## Beregne
Denne noten har forløpig eksempler for funksjonene <b>avlede variabler</b> og <b>fordelingsberegninger</b>


#### Importerer bibliotekene Daplas python pakke (dapla) og pandas.

In [ ]:
import dapla as dp
import pandas as pd

### Avlede variabler

#### Eksempel: 

Summere variabelene x og y fra et datasettet "df" til en ny variabel z:
<code>df['z'] = df['x'] + df['y'] 

avlede variabel "a" ved å dele variabel "x" på variabel "y":
<code>df ['a'] = df['x']/df['y']
  
I eksemplet under lager vi en ny variabel <b>Befolkningstetthet</b> fra variablene <i>innbyggerantall</i> og <i>areal</i>, men først må vi koble datasettene "df_innbyggerantall" og "df_areal" sammen til ett datasett.

### Hente eksempeldata

#### Kjører metoden <i>show</i> for å få oversikt over hvilke parquetdatasett som er tilrettelagt i tilknytning veilderens lagringsområde i sky. Oversikt blir lest inn i egen dataframe og blir kalt <i>df_datasets</i>.
Aktuelt lagringsområde blir lagt inn som parameter (stringobjekt som vi definerer selv) INNDATA_PATH .

In [ ]:
INNDATA_PATH = '/felles/veiledning/pyspark/eksempler/'
df_datasets = dp.show(INNDATA_PATH)


#### df_datasets skrives ut i output vindu. 
Setter opsjonen max_colwith til 150 slik at vi får se all informasjon i df_datasets i output.<br/>
Oversikt over ulike options som kan settes i Pandas:https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html

In [ ]:
pd.options.display.max_colwidth = 150
df_datasets

#### Avleder variabel filname fra variabelpath

In [ ]:
df_datasets["filename"]= df_datasets["path"].str[len(INNDATA_PATH):]
df_datasets

#### Selekterer ut rader bestående av de filnavn som vi ønsker å trekke ut for videre bearbeiding. 
Viser hvordan man kan bruke liste (python list) for å gjøre en slik seleksjon.

In [ ]:
filenames = ['areal', 'innbyggerantall/2020']
df_datasets = df_datasets[df_datasets.filename.isin(filenames)]
df_datasets

#### Oppretter dictionaryet dict_df og leser så de tre parquet datasettene i output vindu over inn som dataframes i dictionaryet. 
Filnavnene blir keys i dictionaryet. Avslutningsvis i pargrafen skrives key-navnene ut i output .

In [ ]:
dict_df = {}
for rows in df_datasets.itertuples():
    dict_df[rows.filename] = dp.read_pandas(rows.path)
for key in dict_df:
    print(key)

### Skriver ut innhold i dataframene i dictionaryet

#### Areal

In [ ]:
dict_df['areal']

#### Innbyggerantall 2020

In [ ]:
dict_df['innbyggerantall/2020']

#### Fjerner duplikater og kolonner vi ikke ønsker å ha med videre
Vi ønsker å sette sammen informasjon fra datasettene i dictinaryet til et nytt datasett. Dette vil vi gjøre ved å koble de sammen via Landkode variabel. Vi fjerner eventuelle dublikater (tar med første forekomst) på Landkode og dropper variable som vi ikke trenger/ønsker å ha med på det nye datasettet i innledende steg under.


Areal

In [ ]:
dict_df['areal'] = dict_df['areal'].drop_duplicates(subset="Land", keep='first', inplace=False).drop(columns= ['Årstall', 'Kilde', 'Landkode'])

Innbyggerantall (bruker liste funksjonalitet)

In [ ]:
liste_drop_kolonner = ['Landkode', 'Kilde', 'År']
dict_df['innbyggerantall/2020'] = dict_df['innbyggerantall/2020'].drop_duplicates(subset="Landkode", keep='first', inplace=False).drop(columns=liste_drop_kolonner)

In [ ]:
# Skriver ut dataframe
dict_df['innbyggerantall/2020']

#### Kobler sammen datasettene og etablerer en dataframe vi skal skrive til dapla for senere å utføre en statistisk analyse på

Kobler først areal og bnp og oppretter dataframen df_areal_innb. Vi legger ikke denne dataframen inn i dictionaryet. Det hadde imidlertid enkelt latt seg gjøre ved å f.eks. bytte ut df_areal_innb med dict['areal_innb].

In [ ]:
df_areal_innb = pd.merge(left=dict_df['areal'], right=dict_df['innbyggerantall/2020'], left_on='Land', right_on='Land')

In [ ]:
df_areal_innb

### Avlede variabelen <i>befolkningstetthet</i>

In [ ]:
df_areal_innb ['Befolkningstetthet'] = df_areal_innb['Innbyggerantall']/df_areal_innb['Areal']
df_areal_innb

#### Lage en grupperingsvariabel
For å vise eksempler på fordelingsberegninger lager vi en grupperingsvariabel på <b>befolkningstetthet</b>. Landene hvor befolkningstetthet er mindre enn 100 plasseseres i gruppe 1, mens de landene med befolkningstetthet på 100 og større plasseres i gruppe 2. 

Dette gjøres ved å benytte en logisk betingelse med funksjonen <code> lambda x </code> https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html

#### Eksempel:
<code>df['avledet variabel'] = df['variabel som er utgangspunktet'].apply(lambda x: 'verdi hvis betingelse er sann' if x betingelse else 'verdi hvis ikke sant')</code>

In [ ]:
df_areal_innb['Tetthetsgruppe'] = df_areal_innb['Befolkningstetthet'].apply(lambda x: "1" if x<100 else "2")

#### Telle opp land i de ulike befolkningstetthetsgruppene

In [ ]:
df_areal_innb['Tetthetsgruppe'].value_counts()

#### Gjennomsnitt og max innbyggertall per Tetthetsgruppe

In [ ]:
df_areal_innb.groupby(['Tetthetsgruppe']).mean().round(2)

In [ ]:
df_areal_innb.groupby(['Tetthetsgruppe']).max()

#### Summere antall innbyggerer

In [ ]:
df_areal_innb['Innbyggerantall'].sum()


#### Telle antall forekomster i kolonnen "Innbyggertall"

In [ ]:
df_areal_innb['Innbyggerantall'].count()
